#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Problem A: Practice writing an optimizing problem
A rectangular box must have volume 500 in^3 (Recall: a box's volume is defined by its $length*width*height$).

Find the shape that has the smallest "mailing length" where the mailing length is defined as the sum of the three edge lengths.

What is the optimal shape of the box that meets this volume requirement with the smallest mailing length possible? (Use ipopt since this is a nonlinear problem).

In [ ]:
#initialize a "Concrete Model"
modelA = ConcreteModel()

#initialize DVs

#define the objective

#(Optional) You can use model.pprint() to see what you've done so far
modelA.pprint()

In [ ]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelA, tee = False) #setting tee = False hides the diagnostic outputs


In [ ]:
#print optimal solution and the smallest mailing length it achieves

#Problem B: Optimizing an Existing Function (office building)
Below, I've included a completed "office building" model as a function. Use Pyomo to solve for the price per square foot in each year that maximizes the total earnings after tax. Use ipopt (since this is a nonlinear problem).

In [ ]:
def office_earnings(total_sqft = 180000,
           m = -0.05,
           b = 1.5,
           op_expense_per_sqft = 1.20,
           heating_surcharge_per_sqft = .2,
           op_exp_annual_growth = .12,
           annual_mortgage = 1500000,
           tax_rate = .34,
           price_per_sqft = [15, 15, 15, 15, 15],
           num_years = 5):
  #rev calc
  perc_occ = [m*price_per_sqft[i] + b for i in range(num_years)]
  sqft_occ = [perc_occ[i]*total_sqft for i in range(num_years)]
  revenue = [sqft_occ[i]*price_per_sqft[i] for i in range(num_years)]
  #operating expense calculations
  base_op_cost_as_percY1 = [(1+op_exp_annual_growth)**i for i in range(num_years)] #note that range(num_years) = range(5) = [0,1,2,3,4] and (1+op_exp_annual_growth)**0 = 1.
  base_op_cost = [op_expense_per_sqft*total_sqft*base_op_cost_as_percY1[i] for i in range(num_years)]
  heating_surcharge = [perc_occ[i]*base_op_cost[i]*heating_surcharge_per_sqft for i in range(num_years)]
  mortgage = [annual_mortgage for i in range(num_years)]
  operating_costs = [base_op_cost[i] + heating_surcharge[i] + mortgage[i] for i in range(num_years)]
  #before and after-tax earnings
  ebt = [revenue[i] - operating_costs[i] for i in range(num_years)]
  taxes = [ebt[i]*tax_rate for i in range(num_years)]
  earnings_after_tax = [ebt[i] - taxes[i] for i in range(num_years)]
  total_earnings_after_tax = sum(earnings_after_tax)
  return total_earnings_after_tax

In [ ]:
#initialize a "Concrete Model"
modelB = ConcreteModel()

#initialize DVs

#define the objective

#(Optional) You can use model.pprint() to see what you've done so far
modelB.pprint()

In [ ]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelB, tee = False) #setting tee = False hides the diagnostic outputs


In [ ]:
#print optimal solution and the largest earnings it achieves

#Problem C: Coding with Lists of Lists and Constraint Lists
Solve this small version of the Stigler problem shown below using lists, `ConstraintLists`, and `for` loops. This version only has **4 decision variables (DVs)** and **3 constraints** but please code in a way that would be scalable for larger problems by following the structure I've started for you below. Print out the optimal \(x\)'s and the total optimal cost.

**Minimize cost =**  
$0.36*x_{\text{wheat}} + 0.141*x_{\text{mac}} + 0.242*x_{\text{cereal}} + 0.300*x_{\text{milk}}$

**Subject to:**  
$
16.1*x_{\text{wheat}} + 1.6*x_{\text{mac}} + 2.9*x_{\text{cereal}} + 12.5*x_{\text{milk}} \geq 3 \quad (\text{Calories Daily Min Constraint})
$
$
7.9*x_{\text{wheat}} + 58.9*x_{\text{mac}} + 91.2*x_{\text{cereal}} + 42.3*x_{\text{milk}} \geq 1.8 \quad (\text{Protein Daily Min Constraint})
$
$
80.5*x_{\text{wheat}} + 3.0*x_{\text{mac}} + 7.2*x_{\text{cereal}} + 15.4*x_{\text{milk}} \geq 2.5 \quad (\text{Fiber Daily Min Constraint})
$

$
x_{\text{wheat}}, x_{\text{mac}}, x_{\text{cereal}}, x_{\text{milk}} \geq 0
$


In [ ]:
#I've put in these input parameters for you
num_commodities = 4    #this is how many food commodities to decide on
num_nutrients = 3      #this is how many nutrient constraints there are
cost_coef = [.36, 0.141, 0.242, 0.300]   #this is a list of the cost coefficients in the objective
constraint_coef = [[16.1, 1.6, 2.9, 12.5],
                   [7.9, 58.9, 91.2, 42.3],
                   [80.5, 3.0, 7.2, 15.4]]     #this is the list of lists of all the constraint coefficients
daily_mins = [3, 1.8, 2.5]        #these are the right hand sides

In [ ]:
#Fill in the ??? to complete this code
modelC = ConcreteModel()

#dvs
modelC.x = Var(range(???), domain = NonNegativeReals)

#objective
modelC.Objective = Objective(expr = ???, sense = ???)

#constraints
modelC.nutrient_constraints = ConstraintList()
for i in range(???):
  modelC.nutrient_constraints.add(???)

#model pprint()
modelC.pprint()

In [ ]:
#solve model with cbc because this is a linear program
opt = SolverFactory('cbc')
results = opt.solve(modelC, tee = True)

In [ ]:
#print optimal amounts of each food and the optimal cost it achieves

